# Lab 3_Mini-project

## The large number of English words can make language-based applications daunting. To cope with this, it is helpful to have a clustering or embedding of these words, so that words with similar meanings are clustered together, or have embeddings that are close to one another.
## But how can we get at the meanings of words? John Firth (1957) put it thus:

## You shall know a word by the company it keeps.

## That is, words that tend to appear in similar contexts are likely to be related. In this assignment, you will investigate this idea by coming up with an embedding of words that is based on co-occurrence statistics.

## The description here assumes you are using Python with NLTK.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import brown
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize 
from collections import Counter
import operator
import collections
import itertools
#from gensim.models import Word2Vec
from nltk.cluster import KMeansClusterer
from sklearn import cluster
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from pylab import rcParams
import warnings
warnings.filterwarnings("ignore")

## • First, download the Brown corpus (using nltk.corpus). This is a collection of text samples from a wide range of sources, with a total of over a million words. Calling brown.words() returns this text in one long list, which is useful.

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/boyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
nltk.corpus.brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [4]:
text = []
text = nltk.corpus.brown.words()
text

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

## • Remove stopwords and punctuation, make everything lowercase, and count how often each word occurs. Use this to come up with two lists:
    – A vocabulary V , consisting of a few thousand (e.g., 5000) of the most commonly-occurring words.
    – A shorter list C of at most 1000 of the most commonly-occurring words, which we shall call context words.

In [5]:
# remove punctuations and digits
text_r = []
for i in text:
    text_r.append(re.sub("[^a-zA-Z]", " ", i))

text_r_1 = list(filter(lambda x: x.isalpha() and len(x) > 1, text_r))
    
df_text = pd.DataFrame(text_r_1, columns=['word'])
df_text

,word
0,The
1,Fulton
2,County
3,Grand
4,Jury
...,...
952520,the
952521,boucle
952522,dress
952523,was


In [6]:
# Convert upper case to lower case
text_r_l = []
for i in range(df_text.word.shape[0]):
    text_r_l.append(df_text.word[i].lower())
text_r_l = pd.DataFrame(text_r_l, columns=['word'])
text_r_l

,word
0,the
1,fulton
2,county
3,grand
4,jury
...,...
952520,the
952521,boucle
952522,dress
952523,was


In [7]:
stopwords = []
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/boyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
filtered_text = [w for w in text_r_l.word if not w in stopwords] 
len(filtered_text)

508631

In [9]:
# count how often each word occurs.
word_count = dict(Counter(filtered_text))
sorted_words = sorted(word_count.items(), key = operator.itemgetter(1), reverse = True)
# first 5000 most commonly-occuring words
V = [x[0] for x in sorted_words[:5000]]
C = V[:1000]

## • For each word w ∈ V , and each occurrence of it in the text stream, look at the surrounding window of four words (two before, two after). Keep count of how often context words from C appear in these positions around word w. That is, for w ∈ V,c ∈ C, define

## n(w, c) = # of times c occurs in a window around w.

## Using these counts, construct the probability distribution Pr(c|w) of context words around w (for each w ∈ V ), as well as the overall distribution Pr(c) of context words. These are distributions over C.

In [10]:
def ls_uniq(seq): 
   checked = []
   for e in seq:
       if e not in checked:
           checked.append(e)
   return checked

c_words = []
for v_word in V:
    four_words = []
    positions = [x for x, n in enumerate(filtered_text) if n == v_word] # locate each word of V in filter_words
    
    for i in positions:
        if i ==0:
            four_word = filtered_text[1:3]
        elif i == 1:
            four_word = ([filtered_text[0]] + filtered_text[2:4])
        else:
            four_word = (filtered_text[(i-2):i] + filtered_text[(i+1):(i+3)])
        four_word_uniq = ls_uniq(four_word)
        four_words = four_words + four_word_uniq
        
    four_words_count = dict(collections.Counter(four_words))
    window_count = len(positions)
    
    for c_word in four_words_count:
        if c_word in C:
            cword_fre = four_words_count[c_word]
            Pr_cw = cword_fre/window_count
            c_words.append((v_word, c_word, cword_fre, window_count, Pr_cw))
            
cwords = pd.DataFrame(c_words)
cwords.columns = ['V_Word','C_Word','Cword_Count','Window_Count','Pr_cw']
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw
0,one,major,12,3292,0.003645
1,one,wanted,8,3292,0.002430
2,one,wait,1,3292,0.000304
3,one,make,33,3292,0.010024
4,one,first,38,3292,0.011543


In [11]:
cwords_uniq = list(cwords['C_Word'].unique())
cwords_pro = {}
for cword in cwords_uniq:
    cwords_pro[cword] = filtered_text.count(cword) / len(filtered_text)
def cword_pro(x):
    return cwords_pro[x]
cwords['Pr_c'] = cwords['C_Word'].apply(cword_pro)
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw,Pr_c
0,one,major,12,3292,0.003645,0.000486
1,one,wanted,8,3292,0.002430,0.000444
2,one,wait,1,3292,0.000304,0.000185
3,one,make,33,3292,0.010024,0.001561
4,one,first,38,3292,0.011543,0.002676


## • Represent each vocabulary item w by a |C|-dimensional vector Φ(w). This is known as the (positive) pointwise mutual information, and has been quite successful in work on word embeddings.

In [12]:
def max_log(row):
    f = row['Pr_cw']
    g = row['Pr_c']
    l = np.math.log(f/g)
    return max(0, l)
cwords['f_w'] = cwords.apply(max_log, axis = 1)
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw,Pr_c,f_w
0,one,major,12,3292,0.003645,0.000486,2.015746
1,one,wanted,8,3292,0.002430,0.000444,1.699134
2,one,wait,1,3292,0.000304,0.000185,0.496933
3,one,make,33,3292,0.010024,0.001561,1.859652
4,one,first,38,3292,0.011543,0.002676,1.461839


In [13]:
mutal_words = pd.pivot_table(cwords, index = 'V_Word', columns = 'C_Word', values = 'f_w')
mutal_words.head()

C_Word,able,accepted,according,account,across,act,action,activities,activity,actual,...,writing,written,wrong,wrote,year,years,yes,yet,york,young
V_Word,,,,,,,,,,,,,,,,,,,,,
abandoned,NaN,NaN,NaN,NaN,NaN,4.275155,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.797542,NaN,NaN
able,3.082068,NaN,NaN,3.002026,NaN,2.118753,NaN,NaN,3.010609,NaN,...,3.002026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.810956
aboard,NaN,NaN,NaN,NaN,4.278695,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
mutal_words = mutal_words.fillna(0)
mutal_words.head()

C_Word,able,accepted,according,account,across,act,action,activities,activity,actual,...,writing,written,wrong,wrote,year,years,yes,yet,york,young
V_Word,,,,,,,,,,,,,,,,,,,,,
abandoned,0.000000,0.0,0.0,0.000000,0.000000,4.275155,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
abel,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
ability,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.797542,0.0,0.000000
able,3.082068,0.0,0.0,3.002026,0.000000,2.118753,0.0,0.0,3.010609,0.0,...,3.002026,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.810956
aboard,0.000000,0.0,0.0,0.000000,4.278695,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [15]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
X = np.asarray(mutal_words)
svd = TruncatedSVD(n_components = 100, n_iter = 7, random_state = 42)
svd.fit(X)
X_reduce = svd.fit_transform(X)
X_reduce_df = pd.DataFrame(X_reduce, index = mutal_words.index)
X_reduce_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
V_Word,,,,,,,,,,,,,,,,,,,,,
abandoned,5.960956,-0.799199,-0.443532,0.477622,0.188357,-0.007875,-1.262637,-0.626547,-0.214247,0.904428,...,1.989921,-1.180750,-0.341614,-0.226722,-0.521254,-0.318842,-0.175451,0.712065,0.108735,0.817915
abel,5.741608,5.306603,1.364197,0.465327,0.085904,-0.433129,-1.344058,1.126805,-1.375314,0.283099,...,1.071310,2.634924,1.321961,0.471964,0.701810,0.080597,-0.825543,-0.675054,0.076028,-0.354717
ability,13.572285,-3.672069,-6.581054,1.495166,0.523935,-1.316302,0.390778,-2.596461,-0.739812,-0.951102,...,-0.415406,-2.488828,3.593068,0.030965,1.542010,0.864126,-1.477301,-1.565202,-2.383625,-1.370630
able,22.168586,-0.288742,-4.411252,-0.904856,2.287316,-2.656532,2.975976,-2.467121,-1.153804,-1.618188,...,-0.099205,0.701866,0.185872,0.171973,1.363279,0.486987,-0.129082,0.754172,-2.233285,0.731055
aboard,6.606274,3.376979,1.393158,-0.672356,-0.731437,0.684725,-1.115608,0.326978,-0.562371,-0.128536,...,-0.838595,1.685687,-2.063308,-0.119458,1.229330,0.077308,-1.512016,-1.404849,1.308275,1.978615


In [16]:
from sklearn.metrics.pairwise import cosine_similarity as cs
words_similarity = 1 - cs(X_reduce, X_reduce)
words_similarity_df = pd.DataFrame(words_similarity, index = 
mutal_words.index,columns = mutal_words.index)
np.fill_diagonal(words_similarity_df.values, 1)
words_similarity_df.head()

V_Word,abandoned,abel,ability,able,aboard,abroad,abrupt,absence,absent,absolute,...,yesterday,yet,yield,york,young,younger,youngsters,youth,zen,zero
V_Word,,,,,,,,,,,,,,,,,,,,,
abandoned,1.000000,0.968543,0.730406,0.533748,0.774131,0.693942,0.911359,0.725871,0.707896,0.616484,...,0.787070,0.586790,0.700104,0.598163,0.675748,0.737834,0.755079,0.592237,0.636707,0.721480
abel,0.968543,1.000000,0.860558,0.759046,0.618207,0.790844,0.780160,0.872887,0.805503,0.957827,...,0.661220,0.574107,0.827330,0.696694,0.576709,0.685788,0.897992,0.634144,0.966296,0.714998
ability,0.730406,0.860558,1.000000,0.464315,0.869166,0.626037,0.544747,0.615985,0.636967,0.680457,...,0.738057,0.422524,0.800745,0.626822,0.512390,0.704118,0.609772,0.531239,0.713440,0.819370
able,0.533748,0.759046,0.464315,1.000000,0.603940,0.513003,0.571522,0.521385,0.553321,0.596704,...,0.676388,0.232921,0.611515,0.443027,0.389125,0.505643,0.553687,0.466592,0.598713,0.753794
aboard,0.774131,0.618207,0.869166,0.603940,1.000000,0.682407,0.842458,0.742596,0.767136,0.803501,...,0.691455,0.576299,0.867090,0.593383,0.490432,0.549534,0.835697,0.614968,0.862326,0.794227


## (b) Nearest neighbor results.

In [17]:
# according to similarity matrix to find the cloest meaning word
words_list = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 'revolution', 'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship']
words_similarity_dict = {}
for word_list in words_list:
    words_similarity_dict[word_list] = words_similarity_df[word_list].idxmin()
for word_dict in words_similarity_dict:
    print ('{} ------> {}'.format(word_dict, words_similarity_dict[word_dict]))

communism ------> century
autumn ------> summer
cigarette ------> wet
pulmonary ------> artery
mankind ------> world
africa ------> asia
chicago ------> portland
revolution ------> world
september ------> december
chemical ------> feed
detergent ------> fabrics
dictionary ------> text
storm ------> weekend
worship ------> community


### Yes, the results make sense!

## (c) Clustering.
Using the vectorial representation Ψ(·), cluster the words in V into 100 groups. Clearly specify what algorithm and distance function you using for this, and the reasons for your choices.
Look over the resulting 100 clusters. Do any of them seem even moderately coherent? Pick out a few of the best clusters and list the words in them.

## Algorithm: KMeansCluster
## Distance function: nltk.cluster.util.cosine_distance
## Reasons: KMeans works iteractively, where initially each centroid is placed randomly in the vector space of the dataset and move themselves to the center of the points which are closer to them. In each new iteration the distance between each centroid and the points are recalculated and the centroids move again to the center of the closest points. The algorithm is finished when the position or the groups don’t change anymore or when the distance in which the centroids change doesn’t surpass a pre-defined threshold.

In [ ]:
kclusterer = KMeansClusterer(100, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X_reduce, assign_clusters=True)

## Yes, some of them seem even moderately coherent.

In [ ]:
# Pick out a few of the best clusters and list the words in them.
strings = []
for index, i in enumerate(X_reduce):    
    strings.append(str(assigned_clusters[index]) + ":" + X_reduce_df.index[assigned_clusters[index]])

print(strings[:10])